In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if bach_first:
        
        - input shape: (bs, len(sq), input_size)
        - hidden shape: (layers * num(directions), bs, hidden_size)
        - output shape: (bs, len(sq), hidden_size * num(directions))
 
 if not bach_first:
 
        - input shape: (len(sq), bs, input_size)
        - hidden shape: (layers * num(directions), bs, hidden_size)
        - output shape: (len(sq), bs, hidden_size * num(directions))

In [2]:
# Three ways to get newaxis
(torch.zeros(3).unsqueeze(0).shape, 
 torch.zeros(3)[None].shape, 
 torch.zeros(3)[np.newaxis, :].shape)

(torch.Size([1, 3]), torch.Size([1, 3]), torch.Size([1, 3]))

In [3]:
bs = 6

In [4]:
lstm = nn.LSTM(input_size=5,
               hidden_size=4,
               batch_first=False,  # !!!!!!!!!
               num_layers=2,
               bidirectional=False)

In [5]:
def cal_params(model):
    params = list(model.parameters())
    params_num = sum(param.numel() for param in params if param.requires_grad)
    print(f'Number of parameters: {params_num}')
    
cal_params(lstm)

Number of parameters: 336


In [6]:
# create input
inputs = torch.randn(3, bs, 5)  # (len(sq), bs, input_size)
inputs, inputs.shape

(tensor([[[-3.5029e-01,  3.4677e-01,  4.2808e-01, -1.6336e-01, -3.3994e-01],
          [-6.0789e-01, -1.3171e+00, -1.6369e+00, -1.6093e+00,  5.8418e-01],
          [-1.3770e+00, -1.3619e+00,  4.3316e-01,  6.5612e-01, -8.2146e-01],
          [-3.8925e-01, -1.3494e+00,  5.6402e-01,  1.3024e+00, -7.5726e-01],
          [ 4.9817e-01,  1.2953e-01,  1.4554e+00, -2.0722e-04, -7.0656e-02],
          [ 3.7583e-01,  5.4918e-01,  3.0456e-01,  1.9616e-01,  7.8082e-01]],
 
         [[-9.1959e-01,  4.6030e-01,  9.9762e-02, -1.0667e-01,  1.3150e-01],
          [ 6.9468e-01, -4.9068e-01, -1.2252e+00, -1.1959e+00,  6.0341e-01],
          [-7.4977e-01, -1.2533e+00,  4.4687e-01, -2.2652e+00, -4.7808e-01],
          [-1.5928e-01, -1.8498e-01,  8.8044e-02, -3.7477e-01, -1.6687e+00],
          [-9.3021e-01,  5.1032e-01,  1.1667e-01, -1.3975e+00,  3.0567e-01],
          [ 9.4562e-01,  9.2504e-02,  1.0332e+00,  1.2200e+00,  2.0081e+00]],
 
         [[ 4.7394e-02,  1.6006e+00, -4.8156e-02, -5.9495e-01, -1.1905

# batch_size = 1

In [12]:
# init hidden
init = torch.zeros(1)[None][None].repeat(2, 1, 4)  
hidden = (init, init)  # (h, c)
init.shape  # (layers * num(directions), bs, hidden_size)

for i in inputs.permute(1 ,0, 2):
    i = i.view(3, 1, 5)  # (len(seq), bs, input_size)
    out, hidden = lstm(i, hidden)
    print(out)
    print(hidden)
    print('\n')
out.shape  # (len(seq), bs, hidden_size * num(directions))

tensor([[[-0.0722,  0.1287, -0.0122,  0.0700]],

        [[-0.0844,  0.1695, -0.0039,  0.0979]],

        [[-0.1049,  0.1927, -0.0052,  0.0907]]], grad_fn=<StackBackward>)
(tensor([[[ 0.0459, -0.0498,  0.1962,  0.3713]],

        [[-0.1049,  0.1927, -0.0052,  0.0907]]], grad_fn=<StackBackward>), tensor([[[ 0.0938, -0.1206,  0.3773,  0.6324]],

        [[-0.1858,  0.7578, -0.0105,  0.1905]]], grad_fn=<StackBackward>))


tensor([[[-0.0666,  0.2245,  0.0061,  0.0661]],

        [[-0.0433,  0.2259,  0.0164,  0.0773]],

        [[-0.0400,  0.2295,  0.0196,  0.1294]]], grad_fn=<StackBackward>)
(tensor([[[ 0.2314,  0.1953,  0.0044,  0.1901]],

        [[-0.0400,  0.2295,  0.0196,  0.1294]]], grad_fn=<StackBackward>), tensor([[[ 0.3296,  0.6045,  0.0077,  0.4178]],

        [[-0.0719,  0.9089,  0.0452,  0.2703]]], grad_fn=<StackBackward>))


tensor([[[-0.0280,  0.2257,  0.0303,  0.1666]],

        [[-0.0055,  0.2109,  0.0617,  0.1478]],

        [[-0.0112,  0.2447,  0.0476,  0.0967]]], grad_fn

torch.Size([3, 1, 4])

# batch_size = bs

In [13]:
init = torch.zeros(1)[None][None].repeat(2, bs, 4)
hidden = (init, init)
h, c = hidden
init.shape

out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)
print('\n')
out.shape, hidden[0].shape

tensor([[[-0.0722,  0.1287, -0.0122,  0.0700],
         [-0.0434,  0.1270, -0.0036,  0.0267],
         [-0.0447,  0.1411, -0.0070,  0.0805],
         [-0.0458,  0.1532, -0.0146,  0.0865],
         [-0.0895,  0.1304, -0.0216,  0.0997],
         [-0.0794,  0.1309, -0.0177,  0.0720]],

        [[-0.0844,  0.1695, -0.0039,  0.0979],
         [-0.0442,  0.1765,  0.0053,  0.0571],
         [-0.0272,  0.1636,  0.0202,  0.0867],
         [-0.0429,  0.2044, -0.0155,  0.1137],
         [-0.0968,  0.1669, -0.0089,  0.0920],
         [-0.1055,  0.1694, -0.0185,  0.1261]],

        [[-0.1049,  0.1927, -0.0052,  0.0907],
         [-0.0466,  0.2032,  0.0097,  0.1153],
         [-0.0327,  0.2140,  0.0135,  0.0563],
         [-0.0647,  0.2082, -0.0173,  0.1170],
         [-0.1050,  0.1864, -0.0037,  0.1184],
         [-0.1118,  0.1919, -0.0138,  0.1352]]], grad_fn=<StackBackward>)
(tensor([[[ 0.0459, -0.0498,  0.1962,  0.3713],
         [ 0.2132,  0.2006,  0.0029,  0.1575],
         [-0.1442, -0.0342, 

(torch.Size([3, 6, 4]), torch.Size([2, 6, 4]))